# Blocky
A blocksim simulation of NaviSim blocks

In [71]:
# Imports
import numpy as np
import pandas as pd
# Import NetworkX
import networkx as nx
# Import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [72]:
def create_traces(G):
	node_trace = create_node_trace(G)
	edge_trace = create_edge_trace(G)
	return edge_trace, node_trace

def create_edge_trace(G):
	edge_x = []
	edge_y = []
	for edge in G.edges():
		x0, y0 = G.nodes[edge[0]]['pos']
		x1, y1 = G.nodes[edge[1]]['pos']
		edge_x.append(x0)
		edge_x.append(x1)
		edge_x.append(None)
		edge_y.append(y0)
		edge_y.append(y1)
		edge_y.append(None)

	edge_trace = go.Scatter(
		x=edge_x, y=edge_y,
		line=dict(width=0.5, color='#888'),
		hoverinfo='none',
		mode='lines')
	return edge_trace

def create_node_trace(G):
	node_x = []
	node_y = []
	for node in G.nodes():
		x, y = G.nodes[node]['pos']
		node_x.append(x)
		node_y.append(y)

	node_trace = go.Scatter(
		x=node_x, y=node_y,
		mode='markers',
		hoverinfo='text',
		marker=dict(
			showscale=True,
			# colorscale options
			#'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
			#'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
			#'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
			colorscale='YlGnBu',
			reversescale=True,
			color=[],
			size=10,
			colorbar=dict(
				thickness=15,
				title='Node Connections',
				xanchor='left',
				titleside='right'
			),
			line_width=2))
	node_types = []
	node_text = []
	# Iterate through nodes
	for node in G.nodes():
		node_types.append(type_chart(G.nodes[node]['type']))
		single_node_text = 'Node: ' + str(node) + '<br>' + 'Type: ' + G.nodes[node]['type'] + '<br>' + 'Compute: ' + str(G.nodes[node]['compute'])
		node_text.append(single_node_text)
	# for node, adjacencies in enumerate(G.adjacency()):
	# 	node_adjacencies.append(len(adjacencies[1]))
	# 	node_text.append('# of connections: '+str(len(adjacencies[1])))
	node_trace.marker.color = node_types
	node_trace.text = node_text
	
	return node_trace

def type_chart(type):
	if type == 'null':
		return 0
	elif type == 'scalar_mult':
		return 1
	elif type == 'ntt':
		return 2
	elif type == 'intt':
		return 3
	elif type == 'point_mult':
		return 4
	else:
		print('Error: unknown type: ', type)
		raise Exception

def plot_graph(G):
	pos = nx.spring_layout(G)
	# Assign positions to nodes
	for node in G.nodes():
		G.nodes[node]['pos'] = list(pos[node])
	edge_trace, node_trace = create_traces(G)
	fig = go.Figure(data=[edge_trace, node_trace],
		layout=go.Layout(
			title='<br>Compute Graph BlockSim',
			titlefont_size=16,
			showlegend=False,
			hovermode='closest',
			margin=dict(b=20,l=5,r=5,t=40),
			annotations=[ dict(
				text="Author: <a href='https://wiki.kaustubh.us'>\
					Kaustubh</a>",
				showarrow=False,
				xref="paper", yref="paper",
				x=0.005, y=-0.002 ) ],
			xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
			yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
		)
	fig.show()

In [73]:
# Read in the data
block_data = pd.read_csv('../data/AMD NTT Profiling - Block Sim.csv')
# Drop empty rows
block_data = block_data.dropna()


In [74]:
# Create metrics DF
metrics = pd.DataFrame(columns=['CUs_busy', 'DRAM_BW', 'L1_BW', 'L2_BW', 'IPC', 'instx', 'DRAM_reads', 'DRAM_writes', 'dram_traffic'])
# Create a list of the metrics
CUs_busy = []
DRAM_BW = []
L1_BW = []
L2_BW = []
IPC = []
instx = []
DRAM_reads = []
DRAM_writes = []
dram_traffic = []


In [75]:
# Functions
# def get_next_node_set()

def create_poly_mult(uid, compute_graph):
	start_node = uid
	compute_graph.add_node(uid, type='null', compute=False)
	uid += 1
	# Add scalar_mult
	scalar_mult_0 = uid
	compute_graph.add_node(uid, type='scalar_mult', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(start_node, scalar_mult_0)
	# Add scalar_mult
	scalar_mult_1 = uid
	compute_graph.add_node(uid, type='scalar_mult', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(start_node, scalar_mult_1)
	# Add ntt
	ntt_0 = uid
	compute_graph.add_node(uid, type='ntt', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(scalar_mult_0, ntt_0)
	# Add ntt
	ntt_1 = uid
	compute_graph.add_node(uid, type='ntt', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(scalar_mult_1, ntt_1)
	# Add point_mult
	point_mult = uid
	compute_graph.add_node(uid, type='point_mult', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(ntt_0, point_mult)
	compute_graph.add_edge(ntt_1, point_mult)
	# Add intt
	intt = uid
	compute_graph.add_node(uid, type='intt', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(point_mult, intt)
	# Add scalar_mult
	scalar_mult_2 = uid
	compute_graph.add_node(uid, type='scalar_mult', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(intt, scalar_mult_2)
	# Add scalar_mult
	scalar_mult_3 = uid
	compute_graph.add_node(uid, type='scalar_mult', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(scalar_mult_2, scalar_mult_3)
	# Add end node
	end_node = uid
	compute_graph.add_node(uid, type='null', compute=False)
	uid += 1
	# Edge
	compute_graph.add_edge(scalar_mult_3, end_node)
	return uid, compute_graph, start_node, end_node




In [34]:
# Create final graph
def final_graph():
	# Create a directed graph
	compute_graph = nx.DiGraph()
	uid = 0
	# Add nodes to the graph
	graph_start_node = uid
	compute_graph.add_node(uid, type='null', compute=False)
	# compute_graph.add_node(uid)
	uid += 1
	graph_end_node = uid
	# compute_graph.add_node(uid)
	compute_graph.add_node(uid, type='null', compute=False)
	uid += 1
	uid, compute_graph, start_node, end_node = create_poly_mult(uid, compute_graph)
	# Add edges to the graph
	compute_graph.add_edge(graph_start_node, start_node)
	compute_graph.add_edge(end_node, graph_end_node)
	return compute_graph

In [35]:
# block_data

In [38]:
compute_graph = final_graph()

In [40]:
plot_graph(compute_graph)

In [69]:
def schedule_compute(compute_graph, ready_list, computing_list, find_next_list, block_data):
	# Schedule the compute
	iterating_ready_list = ready_list.copy()
	for node in iterating_ready_list:
		# Check if the node is in the computing list
		if node in computing_list:
			print('Error: Node already in computing list')
		if gpu_ready_for(node, computing_list):
			print('Pushing node ' + str(node) + ' to computing list')
			computing_list.append(node)
			ready_list.remove(node)
		else:
			print('Stalling, GPU not ready for node' + str(node))
	return compute_graph, ready_list, computing_list, find_next_list

def gpu_ready_for(node, computing_list):
	if len(computing_list) < 2:
		return True
	else:
		return False

def gpu_compute(compute_graph, ready_list, computing_list, find_next_list, block_data, tick):
	# Compute the nodes
	iterating_computing_list = computing_list.copy()
	for node in iterating_computing_list:
		print('Computing node ' + str(node))
		# Set compute to true
		compute_graph.nodes[node]['compute'] = True
		# Move node to find next list
		# print('Moving node ' + str(node) + ' to find next list')
		find_next_list.append(node)
		# Remove node from computing list
		computing_list.remove(node)
	return compute_graph, ready_list, computing_list, find_next_list

def find_next(compute_graph, ready_list, computing_list, find_next_list, block_data):
	# Iterate through the find next list
	iterating_find_next_list = find_next_list.copy()
	for node in iterating_find_next_list:
		# Check if node is computed
		if compute_graph.nodes[node]['compute'] == True:
			# Iterate over children
			children = list(compute_graph.successors(node))
			for child in children:
				if child not in ready_list and child not in computing_list:
					# Make sure child's parents are computed
					parents = list(compute_graph.predecessors(child))
					parents_computed = True
					for parent in parents:
						if compute_graph.nodes[parent]['compute'] == False:
							parents_computed = False
					if parents_computed:
						print('Pushing node ' + str(child) + ' to ready list')
						ready_list.append(child)
			# Remove node from find next list
			# If child's parents are not computed, then the parent who is not computed will add the child to the ready list
			find_next_list.remove(node)
	return compute_graph, ready_list, computing_list, find_next_list

def print_uncomputed_nodes(compute_graph):
	uncomputed_nodes = []
	for node in compute_graph.nodes:
		if compute_graph.nodes[node]['compute'] == False:
			uncomputed_nodes.append(node)
	print('Uncomputed nodes: ' + str(uncomputed_nodes))

def sim_run(compute_graph, block_data):
	# Run till end
	start_node = 0
	# Get the end node
	end_node = 1
	ready_list = []
	computing_list = []
	find_next_list = []
	tick = 0
	break_counter = 100
	ready_list.append(start_node)
	while True:
		print('Tick: ' + str(tick))
		print('Ready list: ' + str(ready_list))
		# print('Computing list: ' + str(computing_list))
		# print('Find next list: ' + str(find_next_list))
		compute_graph, ready_list, computing_list, find_next_list = schedule_compute(compute_graph, ready_list, computing_list, find_next_list, block_data)
		compute_graph, ready_list, computing_list, find_next_list = gpu_compute(compute_graph, ready_list, computing_list, find_next_list, block_data, tick)
		compute_graph, ready_list, computing_list, find_next_list = find_next(compute_graph, ready_list, computing_list, find_next_list, block_data)
		tick += 1
		break_counter -= 1
		if break_counter <= 0:
			break
		# If all 3 lists are empty, then break
		if len(ready_list) == 0 and len(computing_list) == 0 and len(find_next_list) == 0:
			break
	print('Final Tick: ' + str(tick))


In [70]:
sim_run(compute_graph, block_data)
print_uncomputed_nodes(compute_graph)

Tick: 0
Ready list: [0]
Pushing node 0 to computing list
Computing node 0
Pushing node 2 to ready list
Tick: 1
Ready list: [2]
Pushing node 2 to computing list
Computing node 2
Pushing node 3 to ready list
Pushing node 4 to ready list
Tick: 2
Ready list: [3, 4]
Pushing node 3 to computing list
Pushing node 4 to computing list
Computing node 3
Computing node 4
Pushing node 5 to ready list
Pushing node 6 to ready list
Tick: 3
Ready list: [5, 6]
Pushing node 5 to computing list
Pushing node 6 to computing list
Computing node 5
Computing node 6
Pushing node 7 to ready list
Tick: 4
Ready list: [7]
Pushing node 7 to computing list
Computing node 7
Pushing node 8 to ready list
Tick: 5
Ready list: [8]
Pushing node 8 to computing list
Computing node 8
Pushing node 9 to ready list
Tick: 6
Ready list: [9]
Pushing node 9 to computing list
Computing node 9
Pushing node 10 to ready list
Tick: 7
Ready list: [10]
Pushing node 10 to computing list
Computing node 10
Pushing node 11 to ready list
Tick: 8